In [ ]:
import os
import glob
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.models import model_from_json

from sklearn.model_selection import KFold

from sweat4science.workspace.Workspace import Workspace
from sweat4science.evaluation.sessionset import MF_sessionset as mfs
import sweat4science as s4s

from s4s_rnn import utils, evaluation

# import sys
# print(sys.path)

In [ ]:
workspace_folder = "/home/minh/workspace/git/rnd/session-data"
ws = Workspace(workspace_folder)
sessions = mfs.ICT_indoor(ws)


In [ ]:
import re

for session in sessions:
    if len(re.findall("slope", str(session))) > 0:
        sessions.remove(session)
    pass

sessions = np.array(sessions)
print("\n".join(map(str, sessions)))


In [ ]:
eval_dict = evaluation.ExperimentEvalutationDict(sessions)
#glob.glob("../train_results/gru*05step*.gz")

In [ ]:
for model_path in glob.glob("../train_results/*.json"):
    match = re.match('.+20161114.+', model_path)
    old_norm = False
    if match is not None:
        old_norm = True
        pass
    eval_dict.add_model_json(model_path, old_norm=old_norm)
    pass

In [ ]:
for weight_path in glob.glob("../train_results/*.h5"):
    match = re.match('.+20161114.+', weight_path)
    old_norm = False
    if match is not None:
        old_norm = True
        pass
    eval_dict.add_weight_file(weight_path, old_norm=old_norm)
    pass

# os.path.exists('../train_results/lstm_indoor_20161205_15step_04in_400hidden_150epoch_session_01_20150219_weights.h5')

In [ ]:
print('\n'.join(eval_dict.model_json.keys()))
print()
eval_dict.evaluate([#'lstm_lookback10_400neurons_oldnorm',
                    #'lstm_lookback05_400neurons_oldnorm',
                    'lstm_lookback10_400neurons',
                    'lstm_lookback05_400neurons',
                    'lstm_lookback15_400neurons',
                    'gru_lookback05_400neurons',
                    'gru_lookback10_400neurons',
                    'gru_lookback15_400neurons'
                    ])
#['lstm_lookback10_400neurons_oldnorm', 'lstm_lookback05_400neurons_oldnorm']

In [ ]:
eval_dict['session_01_20150304'].plot_predictions([#'lstm_lookback05_400neurons_oldnorm',
                                                   'lstm_lookback15_400neurons',
                                                   #'gru_lookback05_400neurons',
                                                   'gru_lookback15_400neurons'
                                                   ],
                                                  'lstm_lookback05_400neurons_oldnorm')

In [ ]:
eval_dict.plot_error_bar_predictions("Some title")

In [ ]:
# plt.boxplot((eval_dict['session_01_20150304'].predictions['lstm_lookback05_400neurons_oldnorm']
#              - eval_dict['session_01_20150304']._true_output), showmeans=True,
#             labels=['lstm_lookback05_400neurons_oldnorm'])
eval_dict.plot_error_box_predictions(prediction_keys=[#'lstm_lookback05_400neurons_oldnorm',
                                                      #'lstm_lookback10_400neurons_oldnorm',
                                                      'lstm_lookback05_400neurons',
                                                      'gru_lookback05_400neurons',
                                                      'lstm_lookback10_400neurons',
                                                      'gru_lookback10_400neurons',
                                                      'lstm_lookback15_400neurons',
                                                      'gru_lookback15_400neurons'],
                                     title="Box plot")

In [ ]:
def evaluate_cross_validation(sessions, evalutations, result_dir, model_name, experiment_name, date_string,
                              time_steps, hidden_neurons, num_epoch, input_dim=4,
                              save_plot=False, time_horizon=None, old_norm=False,
                              plot_input=False):
    if old_norm:
        print("\n---- Evaluating with old normalization technique:")
        pass
    # Construct base name
    base_name = "%s_%s_%s_%02dstep_%02din_%03dhidden_%03depoch_" \
                % (model_name, experiment_name, date_string, time_steps, input_dim, hidden_neurons, num_epoch)

    base_name = os.path.join(result_dir, base_name)
    print("\nBase name: %s\n" % (base_name))

    # Open model
    model_file_name = base_name  + "model.json"
    json_file = open(model_file_name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()

    # Cross validation testing
    test_predictions = None
    test_actual_outputs = None
    kf = KFold(len(sessions))
    for train_index, test_index in kf.split(sessions):
        test_sessions = sessions[test_index]
        print("\nTesting on:\n" + "\n".join(map(str, test_sessions)))

        test_data_x, test_data_y = \
            utils.get_data_from_sessions(test_sessions, time_steps, return_norm=False, old_norm=old_norm)

        loaded_model = model_from_json(loaded_model_json)

        match = re.match('.+/running_indoor_(.+)/(\d+)>', str(test_sessions[0]))
        cross_validation_name = base_name + match.groups()[0] + "_" + match.groups()[1] + "_"
        session_name = "%s_%s" % (match.groups()[0], match.groups()[1])

        if session_name not in evalutations:
            evalutations[session_name] = evaluation.ExperimentEvalutation(session=test_sessions[0])
            pass

        prediction_name = "%s_lookback%02d_%dneurons" % (model_name, time_steps, hidden_neurons)
        if old_norm:
            prediction_name += "_oldnorm"
            pass

        if time_horizon is not None:
            prediction_name += "_%dhorizon" % time_horizon
            pass

        prediction = utils.evaluate_model(loaded_model, cross_validation_name + "weights.h5",
                                          test_data_x, test_data_y, horizon=time_horizon)
        evalutations[session_name].add_prediction(prediction_name, prediction, unnormalize=True, old_norm=old_norm)

        print("MSE: %.5f\nRMSE: %.5f" % (evalutations[session_name].mse[prediction_name],
                                         np.sqrt(evalutations[session_name].mse[prediction_name])))

        prediction = evalutations[session_name].predictions[prediction_name]
        test_predictions = prediction if test_predictions is None \
            else np.append(test_predictions, prediction, axis=0)
        true_output = evalutations[session_name].true_output[-len(prediction):]
        test_actual_outputs = true_output if test_actual_outputs is None \
            else np.append(test_actual_outputs, true_output, axis=0)

        pass

    mse = np.mean((test_predictions - test_actual_outputs)**2)
    rmse = np.sqrt(mse)
    print("\nOverall results:\n MSE: %.5f\n RMSE: %.5f" % (mse, rmse))

    return #evalutations


In [ ]:
eval_dict = {}
evaluate_cross_validation(sessions=sessions, evalutations=eval_dict, result_dir="../train_results", model_name="lstm",
                          experiment_name="indoor", date_string="20161205", time_steps=5, hidden_neurons=400,
                          num_epoch=150, time_horizon=None)
# evaluate_cross_validation(sessions=sessions, evalutations=eval_dict, result_dir="../train_results", model_name="lstm",
#                           experiment_name="indoor", date_string="20161205", time_steps=10, hidden_neurons=400,
#                           num_epoch=150, time_horizon=None)
# evaluate_cross_validation(sessions=sessions, evalutations=eval_dict, result_dir="../train_results", model_name="lstm",
#                           experiment_name="indoor", date_string="20161205", time_steps=15, hidden_neurons=400,
#                           num_epoch=150, time_horizon=None)

In [ ]:
for exp_name, exp_val in eval_dict.items():
    plot_title = "Heart rate simulation for running_indoor_"
#     plot_title = "Heart rate prediction at %ds horizon for running_indoor_" % time_horizon

    # list(map(a.get, ('1', '3')))
    plot_title += exp_name
    exp_val.plot_predictions(sorted(exp_val.predictions.keys()), plot_title)
    # utils.plot_inputs(test_data_x[:, -1, :])
    pass


In [ ]:
evaluate_cross_validation(sessions=sessions, evalutations=eval_dict, result_dir="../train_results", model_name="gru",
                          experiment_name="indoor", date_string="20161209", time_steps=5, hidden_neurons=400,
                          num_epoch=150, time_horizon=30, old_norm=False)

In [ ]:
for exp_name, exp_val in eval_dict.items():
    plot_title = "Heart rate simulation for running_indoor_"
#     plot_title = "Heart rate prediction at %ds horizon for running_indoor_" % time_horizon

    # list(map(a.get, ('1', '3')))
    plot_title += exp_name
    exp_val.plot_predictions(['lstm_lookback10_400neurons', 'lstm_lookback10_400neurons_oldnorm'], plot_title)
#     exp_val.plot_predictions(['gru_lookback05_400neurons_30horizon'], plot_title)
    # utils.plot_inputs(test_data_x[:, -1, :])
    pass

In [ ]:
eval_dict = {}
evaluate_cross_validation(sessions=sessions, evalutations=eval_dict, result_dir="../train_results", model_name="lstm",
                          experiment_name="indoor", date_string="20161205", time_steps=5, hidden_neurons=400,
                          num_epoch=150, time_horizon=None)
evaluate_cross_validation(sessions=sessions, evalutations=eval_dict, result_dir="../train_results", model_name="gru",
                          experiment_name="indoor", date_string="20161209", time_steps=5, hidden_neurons=400,
                          num_epoch=150, time_horizon=None)

In [ ]:
for exp_name, exp_val in eval_dict.items():
    plot_title = "Heart rate simulation for running_indoor_"
#     plot_title = "Heart rate prediction at %ds horizon for running_indoor_" % time_horizon

    # list(map(a.get, ('1', '3')))
    plot_title += exp_name
    utils.plot_predictions(list(exp_val.predictions.values()), list(exp_val.predictions.keys()),
                           exp_val.true_output, "result.png", plot_title)
    # utils.plot_inputs(test_data_x[:, -1, :])
    pass

In [ ]:
data = np.genfromtxt("artificial_data/sinx_plus_x.csv", delimiter=',')
plt.figure(figsize=(10, 7))
plt.title("Artificially generated data")
plt.plot(data[:, -1:], 'g*')
plt.show()

data_mean = np.mean(data, axis=0)
data_std = np.std(data, axis=0)
data = (data - data_mean) / data_std

num_train = int(0.9*len(data))
test_data_x_ = data[num_train:, :-1]
test_data_y = data[num_train:, -1:]

input_dim = 2
output_dim = 1
hidden_neurons = 400
num_epoch = 50
for ntsteps in [5, 10, 15]:
    base_name = "lstm_sinx_plus_x_" + str(ntsteps) + "step_" + str(input_dim) +\
                "in_" + str(hidden_neurons) + "hidden_" + "20161116" +\
                "_" + str(num_epoch) + "epoch_"
    base_name = os.path.join("train_results", base_name)
    model_file_name = base_name  + "model.json"
    json_file = open(model_file_name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()

    loaded_model = model_from_json(loaded_model_json)

    test_data_x = utils.reshape_array_by_time_steps(test_data_x_, time_steps=ntsteps)
    
    test_data_y_unnormed, test_predict_unnormed, mse = \
            utils.evaluate_model(loaded_model, base_name + "weights.h5",
                                 test_data_x, test_data_y, data_mean[-1], data_std[-1])

    print("MSE: %.5f\nRMSE: %.5f" % (mse, np.sqrt(mse)) )

    # Plot
    utils.plot_predictions(test_predict_unnormed, test_data_y_unnormed, base_name + "result.png",
                           "Artificial data of function sin(x) + x, lookback of " + str(ntsteps), y_label="Output")
    pass

